Author: Alkistis Pourtsidou, ICG Portsmouth
Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [25]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [2]:
import pylab as pl

In [3]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [4]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [5]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [6]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

In [7]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [8]:
#calculate results for these parameters
results = camb.get_results(pars)

#### We are going to work with the linear power spectrum (we won't use non-linear scales)

In [9]:
#Get matter power spectrum at z=0: P(k,z=0)
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=2.0, npoints = 200)

In [10]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

#### IM experiment with redshift range 0.1 < z < 1.45

In [11]:
#Redshift bins
zlist = np.arange(0.1,1.45,0.1)
ztest = zlist[4]
Nzbins = len(zlist)

print (zlist)
print ("ztest =", ztest)
print ("Number of redshift bins =", Nzbins)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4]
ztest = 0.5
Number of redshift bins = 14


#### Useful functions to get the power spectrum in different z

In [12]:
#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))

#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)

print (rcom(ztest))

1946.7257776169436


In [13]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

print (fg(ztest))

0.756249102937079


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return fg(zz)/(1+zz)
def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

print (Dgz(ztest))

0.7701490452523118


#### Fiducial HI abundance and bias fitting functions from SKA Cosmology Red Book 2018

In [15]:
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#### Construct  matter power spectrum P(k,z) - no RSDs

In [16]:
def Pkz(kk,zc):    
    return pow(Dgz(zc),2)*Pkz0(kk)

#### Mean brightness temperature fitting function from SKA Cosmology Red Book 2018

In [17]:
def Tb(zc): #in mK
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

print (ztest, Tb(ztest))

0.5 0.1661


#### Construct HI power spectrum - no RSDs

In [18]:
#Construct P_HI(k,z) [mK^2]
def PHI(kk,zc):
    return pow(Tb(zc),2)*pow(bHI(zc),2)*Pkz(kk,zc)

#### Assume a 4000 square degrees / 4000 hrs IM survey with MeerKAT and construct the noise power spectrum

In [19]:
Ndishes=64
Ddish=13.5*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

Area=4000.0 #deg^2
omegatot = Area*pow(pi/180,2)
ttotal = 4000*60*60*(Area/4000) #4000 hrs for 4000 deg^2

def fc(zc):
    return 1420.4/(1+zc)

def Tsky(zc):
    return 60*pow(300/fc(zc),2.55)*1e3

#receiver temperature from specs document
Tsyslist = [23.5*1e3,23.0*1e3,23.0*1e3,28.0*1e3,29.0*1e3,30.0*1e3,28.5*1e3,29.5*1e3,\
            31.0*1e3,33.0*1e3,34.0*1e3,35.0*1e3,37.0*1e3,38.0*1e3] #mK

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1
dfpix = 50*1e3 #Hz
midfreq = 1420.4e6 #Hz

def dzpix(zc):
    return pow(1+zc,2)*dfpix/midfreq
def sigpix(zc,Tsys):
    return Tsys/np.sqrt(dfpix*tobs(zc)) 
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

def Wsq(kk,zc): #very rough modelling, it should be Wsq(mu,kk,zc) - exercise!
    return np.exp(-pow(kk,2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

def Pnoise(kk,zc,Tsys):
    return pow(sigpix(zc,Tsys),2)*Vpix(zc)*pow(Wsq(kk,zc),-1.)

#### Define minimum and maximum k

In [20]:
def kmin(zc):
    return 2*pi/np.sqrt(pow(rcom(zc),2)*omegatot)
def kmax(zc):
    return 0.14*pow(1+zc,2/(2+nss)); #non-linear cutoff Smith et al 2003

print (round(kmin(ztest),3), round(kmax(ztest),2))

0.003 0.18


#### Calculate effective volume for Fisher Matrix

In [21]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

#effective volume going in the Fisher matrix
def Veff(kk,zc):
    return Vsur(zc)*(PHI(kk,zc)/(PHI(kk,zc)+Pnoise(kk,zc,Tsys)))**2

print ("%.4g" % Vsur(ztest))

1.554e+09


#### Fisher matrix derivatives

In [22]:
def dlnP_dlnOmHIbHI(kk,zc):
    return 2.0

#### Fisher matrix

In [23]:
def dFdk(kk):
    return (1./(4*pi*pi))*pow(kk,2)*pow(dlnP_dlnOmHIbHI(kk,zc),2)*Veff(kk,zc)

#### Get the constraints

In [24]:
for i in range(0,Nzbins):
    zc = round(zlist[i],2)
    Tsys = Tsyslist[i]+Tsky(zc)
    K = np.linspace(kmin(zc), kmax(zc), 100)
    dF = dFdk(K)
    Fisher = scipy.integrate.simps(dF,K) 
    #since we only vary one parameter, we don't need the covariance matrix
    print (zc, round(np.sqrt(1/Fisher),3))

0.1 0.01
0.2 0.005
0.3 0.005
0.4 0.007
0.5 0.009
0.6 0.011
0.7 0.013
0.8 0.015
0.9 0.018
1.0 0.022
1.1 0.026
1.2 0.03
1.3 0.036
1.4 0.041


#### Tonale 2018